In [1]:
import pandas as pd
import numpy as np

import nltk
#from bs4 import BeautifulSoup

import gensim
from gensim import utils
import sys
from sklearn.datasets import fetch_20newsgroups
from nltk import word_tokenize
from nltk import download
from nltk.corpus import stopwords
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

%matplotlib inline
import matplotlib.pyplot as plt


#import os
#import urllib
#import json

#import datetime
#import csv
#import time

#from io import BytesIO

In [46]:
from gensim.models.keyedvectors import KeyedVectors

model = KeyedVectors.load_word2vec_format('./words/GoogleNews-vectors-negative300.bin.gz', binary=True)
#model.save_word2vec_format('./words/GoogleNews-vectors-negative300.txt', binary=False)

In [47]:
download('punkt') #tokenizer, run once
download('stopwords') #stopwords dictionary, run once
stop_words = stopwords.words('english')

[nltk_data] Downloading package punkt to /Users/wjkim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/wjkim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [111]:
# Excel on Mac saves csv file with delimiter ','
# Because there are many comma in texts, you should change them into other characters like space ' '
# I changed comma ',' in text to space' '
photo_posts = pd.read_csv('posts_with_text2.csv')

In [112]:
photo_posts['message'].isnull().any()

True

In [113]:
null_mask = photo_posts['message'].isnull()

idx = np.where(null_mask)
idx
#photo_posts[null_mask]['message']
for i in idx:
    photo_posts.loc[i, 'message'] = ''



#In [174]: mask = x[0]>0
#In [175]: mask
#Out[175]: array([False,  True, False,  True, False], dtype=bool)
##In [176]: idx = np.where(mask)[0]
#In [177]: idx
#Out[177]: array([1, 3], dtype=int32)
#In [178]: x[0, idx[1]]
#Out[178]: 6.0

photo_posts['message'].isnull().any()

False

In [114]:
photo_posts['text'].isnull().any()

True

In [115]:
null_mask = photo_posts['text'].isnull()

idx = np.where(null_mask)
idx
#photo_posts[null_mask]['message']
for i in idx:
    photo_posts.loc[i, 'text'] = ''



#In [174]: mask = x[0]>0
#In [175]: mask
#Out[175]: array([False,  True, False,  True, False], dtype=bool)
##In [176]: idx = np.where(mask)[0]
#In [177]: idx
#Out[177]: array([1, 3], dtype=int32)
#In [178]: x[0, idx[1]]
#Out[178]: 6.0

photo_posts['text'].isnull().any()

False

In [116]:
messages = photo_posts['message'].tolist()
texts = photo_posts['text'].tolist()
y = photo_posts['label'].tolist()

In [117]:
def preprocess(text):
    text = text.lower()
    doc = word_tokenize(text)
    doc = [word for word in doc if word not in stop_words]
    doc = [word for word in doc if word.isalpha()] #restricts string to alphabetic characters only
    return doc

In [121]:
#messages

In [122]:
len(messages)

206

In [123]:
t_corpus = [preprocess(str(text)) for text in texts]

In [124]:
m_corpus = [preprocess(str(message)) for message in messages]
len(m_corpus)

206

In [125]:
def filter_docs(corpus, texts, labels, condition_on_doc):
    """
    Filter corpus, texts and labels given the function condition_on_doc which takes
    a doc.
    The document doc is kept if condition_on_doc(doc) is true.
    """
    number_of_docs = len(corpus)

    #if texts is not None:
    #    texts = [text for (text, doc) in zip(texts, corpus)
    #             if condition_on_doc(doc)]

    #labels = [i for (i, doc) in zip(labels, corpus) if condition_on_doc(doc)]
    #corpus = [doc for doc in corpus if condition_on_doc(doc)]
    
    if texts is not None:
        for i in range(number_of_docs):
            doc = corpus[i]
            if condition_on_doc(doc) is False or doc == 'nan':
                texts[i] = ''
                corpus[i] = []
                

    print("{} docs removed".format(number_of_docs - len(corpus)))

    return (corpus, texts, labels)

In [126]:
#t_corpus, texts, y = filter_docs(t_corpus, texts, y, lambda doc: (len(doc) != 0))

In [127]:
#m_corpus, messages, y = filter_docs(m_corpus, messages, y, lambda doc: (len(doc) != 0))

In [128]:
# Remove OOV words and documents with no words in model dictionary
def document_vector(word2vec_model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in word2vec_model.vocab]
    return np.mean(word2vec_model[doc], axis=0)

In [129]:
def has_vector_representation(word2vec_model, doc):
    """check if at least one word of the document is in the
    word2vec dictionary"""
    return not all(word not in word2vec_model.vocab for word in doc)

In [130]:
t_corpus, texts, y = filter_docs(t_corpus, texts, y, lambda doc: has_vector_representation(model, doc))
#t_corpus

0 docs removed


In [131]:
m_corpus, messages, y = filter_docs(m_corpus, messages, y, lambda doc: has_vector_representation(model, doc))
#m_corpus

0 docs removed


In [133]:
x1 =[]
for doc in t_corpus: #look up each doc in model
    #print("i: ", i)
    #i = i + 1
    if len(doc) == 0:
        x1.append(np.zeros((300,)))
    else:
        x1.append(document_vector(model, doc))

In [134]:
t_corpus[25]

['months', 'republicanparty', 'need', 'phones', 'back']

In [135]:
len(t_corpus[25])

5

In [136]:
len(m_corpus)

206

In [137]:
m_corpus[25]

[]

In [138]:
len(m_corpus[25])

0

In [139]:
document_vector(model, t_corpus[25]).shape

(300,)

In [140]:
#document_vector(model, m_corpus[25]).shape

In [141]:
x2 =[]
i = 0
for doc in m_corpus: #look up each doc in model
    #print("i: ", i)
    #i = i + 1
    if len(doc) == 0:
        x2.append(np.zeros((300,)))
    else:
        x2.append(document_vector(model, doc))

In [142]:
cosine_similarity(x1[0].reshape(1,300), x2[0].reshape(1,300))

array([[ 0.]])

In [143]:
m_corpus[80]

[]

In [144]:
t_corpus[80]

['confuse', 'dying', 'oil', 'fighting', 'freedom']

In [145]:
sim_list =[]
#i = 0
for (x1i, x2i) in zip(x1, x2):
    #print(x[i], y[i])
    x1i = x1i.reshape(1,300)
    x2i = x2i.reshape(1,300)
    sim = cosine_similarity(x1i, x2i)
    sim_list.append(sim[0,0])
    print(sim[0,0])
    #i = i + 1

0.0
0.612595
0.482248
0.877566
0.858846
0.840662
0.797358
0.829173
0.906689
0.916016
0.650828
0.875855
0.482248
0.728008
0.669229
0.0
0.0
0.71568
0.0
0.782292
0.55103
0.55103
0.524588
0.0
0.672423
0.0
0.0
0.169928
0.0
0.211118
0.0
0.0
0.0
0.209111
0.0
0.0
0.0
0.11615
0.179205
0.172701
0.209539
0.0
0.0
0.0
0.0
0.196493
0.203213
0.225163
0.129964
0.130402
0.0
0.0
0.0
0.162752
0.184658
0.17189
0.187886
0.166878
0.207407
0.0
0.0
0.0
0.0
0.0
0.116773
0.0
0.0
0.0
0.130959
0.168639
0.204695
0.484077
0.512756
0.655675
0.580829
0.550359
0.530698
0.616639
0.392135
0.529577
0.0
0.721297
0.680437
0.521895
0.513004
0.695907
0.715502
0.69992
0.0
0.0
0.321836
0.0
0.374382
0.0991875
0.177421
0.376653
0.555816
0.725141
0.624006
0.722035
0.59488
0.126407
0.563882
0.567024
0.315656
0.153201
0.521011
0.331885
0.549289
0.521072
0.5157
0.0
0.288622
0.177582
0.562688
0.248976
0.196631
0.524225
0.573337
0.637336
0.567582
0.347329
0.0
0.569371
0.571626
0.740561
0.715641
0.656002
0.611722
0.616726
0.522578
0.69

In [149]:
sim_list

[0.0,
 0.61259508,
 0.48224804,
 0.87756634,
 0.85884625,
 0.84066164,
 0.79735804,
 0.82917309,
 0.90668917,
 0.91601622,
 0.65082794,
 0.87585545,
 0.48224804,
 0.72800839,
 0.66922855,
 0.0,
 0.0,
 0.71568,
 0.0,
 0.78229225,
 0.55103028,
 0.55103028,
 0.52458793,
 0.0,
 0.67242283,
 0.0,
 0.0,
 0.16992766,
 0.0,
 0.21111773,
 0.0,
 0.0,
 0.0,
 0.20911056,
 0.0,
 0.0,
 0.0,
 0.11615004,
 0.17920546,
 0.17270111,
 0.20953906,
 0.0,
 0.0,
 0.0,
 0.0,
 0.19649294,
 0.20321271,
 0.22516277,
 0.12996365,
 0.13040234,
 0.0,
 0.0,
 0.0,
 0.16275159,
 0.18465787,
 0.17189024,
 0.18788595,
 0.16687775,
 0.20740703,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.11677337,
 0.0,
 0.0,
 0.0,
 0.13095862,
 0.16863933,
 0.20469536,
 0.48407704,
 0.51275599,
 0.65567493,
 0.58082932,
 0.55035949,
 0.53069758,
 0.61663908,
 0.39213455,
 0.52957654,
 0.0,
 0.72129703,
 0.68043697,
 0.52189499,
 0.51300383,
 0.69590735,
 0.71550173,
 0.69992042,
 0.0,
 0.0,
 0.32183564,
 0.0,
 0.37438226,
 0.099187523,
 0.17742106

In [150]:
FILE_READ_SIZE = 1024

In [151]:
# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)

dog = model['dog']
print(dog.shape)
print(dog[:10])

# Deal with an out of dictionary word: Михаил (Michail)
if 'Михаил' in model:
    print(model['Михаил'].shape)
else:
    print('{0} is an out of dictionary word'.format('Михаил'))

# Some predefined functions that show content related information for given words
print("similarity\n")
print(model.most_similar(positive=['woman', 'king'], negative=['man']))

print(model.doesnt_match("breakfast cereal dinner lunch".split()))

print(model.similarity('woman', 'man'))

(300,)
[ 0.05126953 -0.02233887 -0.17285156  0.16113281 -0.08447266  0.05737305
  0.05859375 -0.08251953 -0.01538086 -0.06347656]
Михаил is an out of dictionary word
similarity

[('queen', 0.7118192911148071), ('monarch', 0.6189674139022827), ('princess', 0.5902431607246399), ('crown_prince', 0.5499460697174072), ('prince', 0.5377321243286133), ('kings', 0.5236844420433044), ('Queen_Consort', 0.5235945582389832), ('queens', 0.5181134343147278), ('sultan', 0.5098593235015869), ('monarchy', 0.5087411999702454)]
cereal
0.7664012231


In [152]:
url = "http://abcn.ws/2cMwUPw"    
html = urllib.request.urlopen(url).read()    

soup = BeautifulSoup(html, 'html.parser')

#print(soup.prettify())
print(soup.get_text())

NameError: name 'urllib' is not defined

In [ ]:
print(html)